In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
import os
import pickle
import seaborn as sns

In [ ]:
def plot_losses(losses, embedding):
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=range(1, len(losses) + 1), y=losses, label='Loss')
    plt.title(f'Loss - {embedding}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    # plt.show()
    plt.savefig(f'Plots/loss_{embedding}.png')

def plot_f1s(f1_scores, embedding):
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=range(1, len(f1_scores) + 1), y=f1_scores, label='F1 Score')
    plt.title(f'F1 Score - {embedding}')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    # plt.show()
    plt.savefig(f'Plots/f1_{embedding}.png')

# Plotting Baseline models

In [ ]:
base_path = 'Part 2/Non Trainable Embeddings'
models = os.listdir(base_path)

for model in models:
    final_path = base_path + '/' + model
    train_losses = pickle.load(open(final_path + '/train_loss.pkl', 'rb'))
    val_losses = pickle.load(open(final_path + '/val_loss.pkl', 'rb'))
    train_f1s = pickle.load(open(final_path + '/train_f1.pkl', 'rb'))
    val_f1s = pickle.load(open(final_path + '/val_f1.pkl', 'rb'))

    plot_losses(train_losses, model)
    plot_f1s(train_f1s, model)
    plot_losses(val_losses, model)
    plot_f1s(val_f1s, model)

# Plotting BiLSTM-CRF

In [ ]:
base_path = 'Part 3/Non Trainable Embeddings'
models = os.listdir(base_path)

for model in models:
    final_path = base_path + '/' + model
    train_losses = pickle.load(open(final_path + '/train_loss.pkl', 'rb'))
    train_losses = [float(loss.item()) for loss in train_losses]

    val_losses = pickle.load(open(final_path + '/val_loss.pkl', 'rb'))
    val_losses = [float(loss.item()) for loss in val_losses]

    train_f1s = pickle.load(open(final_path + '/train_f1.pkl', 'rb'))
    train_f1s = [float(f1.item()) for f1 in train_f1s]

    val_f1s = pickle.load(open(final_path + '/val_f1.pkl', 'rb'))
    val_f1s = [float(f1.item()) for f1 in val_f1s]

    plot_losses(train_losses, f'BiLSTM CRF + {model}')
    plot_f1s(train_f1s, model)
    plot_losses(val_losses, model)
    plot_f1s(val_f1s, model)